# Dataton BC 2018

*Dirección de Capacidades Analíticas y Gobierno de Información, Grupo Bancolombia*<br>
Esta versión: *2018-10-17*<br>
Documentación tablas

## Naturaleza de los datos

Los datos entregados en este reto corresponden a transacciones realizadas por clientes persona del banco vía [PSE](https://www.pse.com.co/inicio). Estas transacciones, a diferencia de las transacciones realizadas vía POS, no cuentan con un código [MCC](https://en.wikipedia.org/wiki/Merchant_category_code) atado a la transacción, que permite conocer la categoría de comercio a la que pertence el establecimiento de comercio donde se realiza la transacción. Adicionalmente, muchas de estas transferencias por PSE corresponden a transferencias de pagos de servicios públicos, seguros, colegios, arrendamientos, y otros gastos que pueden ser denominados como gastos grandes. En el marco de un sistema de gestión de finanzas personales, poder categorizar adecuadamente estas transacciones que se realizan por PSE es de suma importancia para contar con una foto completa de la actividad de gastos de los clientes. Para este reto, los equipos participantes tendrán acceso a una muestra de transacciones PSE que corresponden a algo más de 300 mil clientes (persona), seleccionados de manera aleatoria. La tabla de transacciones cuenta con 11.8 millones de registros (uno para cada transacción), realizados entre septiembre de 2016 y octubre de 2018.

**NOTA** Los datos han pasado por un proceso relativamente simple de curación, pero se han dejado algunos ruidos en la calidad de éstos con el fin de que los equipos también lleven a cabo un proceso de inspección y limpieza.


## NOTA IMPORTANTE

Recuerden que esta información aún contiene un elevado nivel de ruido. No solo no ha sido depurada de posibles datos atípicos (transacciones de valor muy elevado) fruto de errores o transacciones fallidas, sino que también cuenta con el ruido asociado al campo de referencia, donde se involucra el factor humano, ya que son campos de texto libre que pueden contener cualquier tipo de información.

Por seguridad, hemos eliminado cualquier número presente en dichos campos de referencia (cédulas, nits, montos, contratos, etc.).


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import random
import time
%matplotlib inline
import matplotlib.animation as animation
import csv
matplotlib.style.use('seaborn')

from sklearn import preprocessing

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn import manifold
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster import hierarchy
from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import pairwise_distances
# from wordcloud import WordCloud

from mpl_toolkits.mplot3d import Axes3D

Cargar los datos proeprocesados de las bases de datos

In [2]:
pagadores=pd.read_csv("info_pagadores.csv",index_col=0)
# pagadores=pagadores.iloc[:,1:]
pagadores.head()

,id_cliente,seg_str,ocupacion,tipo_vivienda,nivel_academico,estado_civil,genero,edad,ingreso_rango
0,18,PERSONAL PLUS,5,O,U,M,M,92.0,e. (4.4 5.5MM]
1,32,PERSONAL PLUS,E,F,T,M,M,80.0,i. (8.7 Inf)
2,41,EMPRENDEDOR,3,O,I,W,M,90.0,b. (1.1 2.2MM]
3,47,EMPRENDEDOR,7,NaN,I,I,M,86.0,c. (2.2 3.3MM]
4,71,PERSONAL,5,O,S,M,M,79.0,e. (4.4 5.5MM]


In [3]:
transac_pag=pd.read_csv("client_fr_sum_periods.csv",index_col=0)
transac_pag.head()

,id_cliente,count,valor_trx_suma,dia_sem_0.0,dia_sem_1.0,dia_sem_2.0,dia_sem_3.0,dia_sem_4.0,dia_sem_5.0,dia_sem_6.0,...,semana_year_sum_44.0,semana_year_sum_45.0,semana_year_sum_46.0,semana_year_sum_47.0,semana_year_sum_48.0,semana_year_sum_49.0,semana_year_sum_50.0,semana_year_sum_51.0,semana_year_sum_52.0,semana_year_sum_53.0
0,1,26,42769383.24,11,6,6,2,1,0,0,...,NaN,1765195.73,NaN,79652.03,NaN,NaN,3395078.84,NaN,NaN,NaN
1,2,6,1365034.29,1,0,2,0,3,0,0,...,NaN,228017.43,NaN,NaN,NaN,NaN,227118.80,NaN,NaN,NaN
2,3,53,12058409.54,4,23,8,6,12,0,0,...,354125.88,362422.61,NaN,NaN,NaN,2376708.76,NaN,346696.94,NaN,NaN
3,4,7,2534093.31,0,0,1,2,4,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,27,10990697.40,3,6,7,6,5,0,0,...,825055.41,NaN,NaN,NaN,449752.12,388489.09,NaN,NaN,NaN,NaN


# Normalizar los datos

In [4]:
transac_pag=pd.merge(transac_pag,pagadores[["id_cliente","edad"]],on='id_cliente')
transac_pag.head()

,id_cliente,count,valor_trx_suma,dia_sem_0.0,dia_sem_1.0,dia_sem_2.0,dia_sem_3.0,dia_sem_4.0,dia_sem_5.0,dia_sem_6.0,...,semana_year_sum_45.0,semana_year_sum_46.0,semana_year_sum_47.0,semana_year_sum_48.0,semana_year_sum_49.0,semana_year_sum_50.0,semana_year_sum_51.0,semana_year_sum_52.0,semana_year_sum_53.0,edad
0,1,26,42769383.24,11,6,6,2,1,0,0,...,1765195.73,NaN,79652.03,NaN,NaN,3395078.84,NaN,NaN,NaN,92.0
1,2,6,1365034.29,1,0,2,0,3,0,0,...,228017.43,NaN,NaN,NaN,NaN,227118.80,NaN,NaN,NaN,87.0
2,3,53,12058409.54,4,23,8,6,12,0,0,...,362422.61,NaN,NaN,NaN,2376708.76,NaN,346696.94,NaN,NaN,89.0
3,4,7,2534093.31,0,0,1,2,4,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,91.0
4,5,27,10990697.40,3,6,7,6,5,0,0,...,NaN,NaN,NaN,449752.12,388489.09,NaN,NaN,NaN,NaN,91.0


## One hot-encoding para los categóricos

In [5]:
pagadores=pagadores[["id_cliente","seg_str","ocupacion","tipo_vivienda","nivel_academico","estado_civil","genero","ingreso_rango"]]
pagadores.index=(pagadores.id_cliente)
pagadores=pagadores.iloc[:,1:]
pagadores.ingreso_rango=pagadores.ingreso_rango.replace(["a. (0  1.1MM]",
                                                         "b. (1.1  2.2MM]",
                                                         "c. (2.2  3.3MM]",
                                                         "d. (3.3  4.4MM]",
                                                         "e. (4.4  5.5MM]",
                                                         "f. (5.5  6.6MM]",
                                                         "g. (6.6  7.6MM]",
                                                         "h. (7.6  8.7MM]",
                                                         "i. (8.7  Inf)"],
                                                        ["a","b","c","d","e","f","g","h","i"])
pagadores.head()

,seg_str,ocupacion,tipo_vivienda,nivel_academico,estado_civil,genero,ingreso_rango
id_cliente,,,,,,,
18,PERSONAL PLUS,5,O,U,M,M,e
32,PERSONAL PLUS,E,F,T,M,M,i
41,EMPRENDEDOR,3,O,I,W,M,b
47,EMPRENDEDOR,7,NaN,I,I,M,c
71,PERSONAL,5,O,S,M,M,e


In [6]:
pagadores_oneh=pd.get_dummies(pagadores).astype("float64")

## Escala percentil de los valores  numéricos

In [7]:
transac_pag.index=(transac_pag.id_cliente)
transac_pag=transac_pag.iloc[:,1:]
transac_pag.head()

,count,valor_trx_suma,dia_sem_0.0,dia_sem_1.0,dia_sem_2.0,dia_sem_3.0,dia_sem_4.0,dia_sem_5.0,dia_sem_6.0,dia_mes_1.0,...,semana_year_sum_45.0,semana_year_sum_46.0,semana_year_sum_47.0,semana_year_sum_48.0,semana_year_sum_49.0,semana_year_sum_50.0,semana_year_sum_51.0,semana_year_sum_52.0,semana_year_sum_53.0,edad
id_cliente,,,,,,,,,,,,,,,,,,,,,
1,26,42769383.24,11,6,6,2,1,0,0,0,...,1765195.73,NaN,79652.03,NaN,NaN,3395078.84,NaN,NaN,NaN,92.0
2,6,1365034.29,1,0,2,0,3,0,0,0,...,228017.43,NaN,NaN,NaN,NaN,227118.80,NaN,NaN,NaN,87.0
3,53,12058409.54,4,23,8,6,12,0,0,0,...,362422.61,NaN,NaN,NaN,2376708.76,NaN,346696.94,NaN,NaN,89.0
4,7,2534093.31,0,0,1,2,4,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,91.0
5,27,10990697.40,3,6,7,6,5,0,0,6,...,NaN,NaN,NaN,449752.12,388489.09,NaN,NaN,NaN,NaN,91.0


In [8]:
transac_pag=transac_pag.fillna(0)
quantile_transformer = preprocessing.QuantileTransformer(random_state=0)
transac_pag_centil= quantile_transformer.fit_transform(transac_pag)
transac_pag_centil

array([[6.11611612e-01, 9.38118892e-01, 8.30830831e-01, ...,
        9.99999998e-08, 9.99999998e-08, 9.99449945e-01],
       [3.26826827e-01, 3.72638619e-01, 3.51351351e-01, ...,
        9.99999998e-08, 9.99999998e-08, 9.98814991e-01],
       [7.83783784e-01, 7.58337709e-01, 6.08108108e-01, ...,
        9.99999998e-08, 9.99999998e-08, 9.99119912e-01],
       ...,
       [7.39239239e-01, 8.35184156e-01, 5.48048048e-01, ...,
        7.89763136e-01, 9.99999998e-08, 6.50650651e-03],
       [9.99999998e-08, 3.53935792e-01, 9.99999998e-08, ...,
        9.99999998e-08, 9.99999998e-08, 6.50650651e-03],
       [2.61261261e-01, 1.97512398e-01, 9.99999998e-08, ...,
        9.99999998e-08, 9.99999998e-08, 6.50650651e-03]])

In [9]:
transac_pag_centil=pd.DataFrame(transac_pag_centil, index=transac_pag.index, columns=transac_pag.columns)
transac_pag_centil=transac_pag_centil.fillna(0)
transac_pag_centil.head()

,count,valor_trx_suma,dia_sem_0.0,dia_sem_1.0,dia_sem_2.0,dia_sem_3.0,dia_sem_4.0,dia_sem_5.0,dia_sem_6.0,dia_mes_1.0,...,semana_year_sum_45.0,semana_year_sum_46.0,semana_year_sum_47.0,semana_year_sum_48.0,semana_year_sum_49.0,semana_year_sum_50.0,semana_year_sum_51.0,semana_year_sum_52.0,semana_year_sum_53.0,edad
id_cliente,,,,,,,,,,,,,,,,,,,,,
1,0.611612,0.938119,8.308308e-01,6.781782e-01,0.685686,4.639640e-01,0.347347,1.000000e-07,1.000000e-07,1.000000e-07,...,9.797657e-01,1.000000e-07,8.208357e-01,1.000000e-07,1.000000e-07,9.882952e-01,1.000000e-07,1.000000e-07,1.000000e-07,0.999450
2,0.326827,0.372639,3.513514e-01,1.000000e-07,0.453954,1.000000e-07,0.547047,1.000000e-07,1.000000e-07,1.000000e-07,...,8.788187e-01,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,8.467177e-01,1.000000e-07,1.000000e-07,1.000000e-07,0.998815
3,0.783784,0.758338,6.081081e-01,9.469469e-01,0.751251,6.956957e-01,0.853854,1.000000e-07,1.000000e-07,1.000000e-07,...,9.076384e-01,1.000000e-07,1.000000e-07,1.000000e-07,9.795904e-01,1.000000e-07,9.000112e-01,1.000000e-07,1.000000e-07,0.999120
4,0.352853,0.470735,1.000000e-07,1.000000e-07,0.333834,4.639640e-01,0.609109,1.000000e-07,1.000000e-07,1.000000e-07,...,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,0.999340
5,0.620621,0.741476,5.480480e-01,6.781782e-01,0.721221,6.956957e-01,0.659159,1.000000e-07,1.000000e-07,9.409409e-01,...,1.000000e-07,1.000000e-07,1.000000e-07,8.956852e-01,8.847695e-01,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,0.999340


In [10]:
clientes_normaliz=pagadores_oneh.join(transac_pag_centil)
clientes_normaliz.head()

,seg_str_EMPRENDEDOR,seg_str_OTRO,seg_str_PERSONAL,seg_str_PERSONAL PLUS,seg_str_PREFERENCIAL,ocupacion_1,ocupacion_2,ocupacion_3,ocupacion_4,ocupacion_5,...,semana_year_sum_45.0,semana_year_sum_46.0,semana_year_sum_47.0,semana_year_sum_48.0,semana_year_sum_49.0,semana_year_sum_50.0,semana_year_sum_51.0,semana_year_sum_52.0,semana_year_sum_53.0,edad
id_cliente,,,,,,,,,,,,,,,,,,,,,
18,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,9.730260e-01,1.000000e-07,1.000000e-07,1.000000e-07,8.893509e-01,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,0.999450
32,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,0.996997
41,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,8.896102e-01,1.000000e-07,1.000000e-07,1.000000e-07,9.073192e-01,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,0.999230
47,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000e-07,8.329252e-01,9.483258e-01,1.000000e-07,8.030368e-01,1.000000e-07,9.445174e-01,8.005076e-01,1.000000e-07,0.998611
71,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.000000e-07,1.000000e-07,9.239337e-01,8.131081e-01,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,0.995996


In [11]:
clientes_normaliz=clientes_normaliz.replace(np.nan,0)
clientes_normaliz.to_csv("clientes_normaliz.csv")